# 🧾 TaxAlly Production Agent - Colab Server

**Production-grade agent designed for seamless frontend/backend integration**

## Architecture
- ✅ Matches exact frontend/backend contracts
- ✅ Structured input/output (no hacks)
- ✅ Session management
- ✅ Same tax logic (all tools preserved)
- ✅ Better error handling

## Setup
1. Runtime → Change runtime type → T4 GPU
2. Run all cells
3. Copy ngrok URL to server/.env

In [1]:
#@title 1️⃣ Install Dependencies
!pip install -q transformers accelerate bitsandbytes torch
!pip install -q fastapi uvicorn pyngrok nest-asyncio
!pip install -q pydantic

print("✅ Dependencies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.7 MB/s eta 0:00:00
✅ Dependencies installed!


In [2]:
#@title 2️⃣ Setup ngrok
# Get token from: https://dashboard.ngrok.com/get-started/your-authtoken

NGROK_AUTH_TOKEN = "2zh80dsdTkMU8LwKIgoHTc83vM1_2SWnKGKPDMDTtPGVqGgaa"  #@param {type:"string"}

if NGROK_AUTH_TOKEN:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok configured!")
else:
    print("⚠️ No ngrok token provided")
    print("Get free token at: https://dashboard.ngrok.com/get-started/your-authtoken")

✅ ngrok configured!


In [16]:
#@title 3️⃣ Load Qwen2.5-7B-Instruct Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

print(f"🔄 Loading {MODEL_NAME}...")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print(f"✅ Model loaded! Memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

🔄 Loading Qwen/Qwen2.5-7B-Instruct...
GPU: Tesla T4


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded! Memory: 5.56 GB


In [17]:
#@title 4️⃣ TaxAlly Tools (Same as before)
from datetime import datetime
from typing import Dict, Any

# Tax slabs (AY 2025-26)
OLD_REGIME_SLABS = [
    {"min": 0, "max": 250000, "rate": 0},
    {"min": 250000, "max": 500000, "rate": 5},
    {"min": 500000, "max": 1000000, "rate": 20},
    {"min": 1000000, "max": float('inf'), "rate": 30}
]

NEW_REGIME_SLABS = [
    {"min": 0, "max": 300000, "rate": 0},
    {"min": 300000, "max": 700000, "rate": 5},
    {"min": 700000, "max": 1000000, "rate": 10},
    {"min": 1000000, "max": 1200000, "rate": 15},
    {"min": 1200000, "max": 1500000, "rate": 20},
    {"min": 1500000, "max": float('inf'), "rate": 30}
]

SPECIAL_STATES = ['Arunachal Pradesh', 'Assam', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Sikkim', 'Tripura']

def calculate_slab_tax(income: float, slabs: list) -> tuple:
    remaining = income
    total_tax = 0
    breakdown = []

    for slab in slabs:
        if remaining <= 0:
            break
        taxable = min(remaining, slab["max"] - slab["min"])
        tax = taxable * slab["rate"] / 100
        if tax > 0:
            breakdown.append({"slab": f"{slab['rate']}%", "tax": tax})
        total_tax += tax
        remaining -= taxable

    return total_tax, breakdown

class TaxAllyTools:
    @staticmethod
    def calculate_income_tax(income: float, deductions_80c: float = 0, deductions_80d: float = 0) -> dict:
        """Calculate income tax with old vs new regime comparison."""
        std_deduction = 75000

        # Old regime
        old_deductions = min(deductions_80c, 150000) + min(deductions_80d, 50000) + std_deduction
        old_taxable = max(0, income - old_deductions)
        old_tax, old_breakdown = calculate_slab_tax(old_taxable, OLD_REGIME_SLABS)
        if old_taxable <= 500000:
            old_tax = max(0, old_tax - 12500)
        old_cess = old_tax * 0.04
        old_total = old_tax + old_cess

        # New regime
        new_taxable = max(0, income - std_deduction)
        new_tax, new_breakdown = calculate_slab_tax(new_taxable, NEW_REGIME_SLABS)
        if new_taxable <= 700000:
            new_tax = max(0, new_tax - 25000)
        new_cess = new_tax * 0.04
        new_total = new_tax + new_cess

        savings = abs(old_total - new_total)
        recommendation = f"{'Old' if old_total < new_total else 'New'} Regime saves ₹{savings:,.0f}"

        return {
            "old_regime": {"totalTax": old_total, "effectiveRate": (old_total/income*100) if income > 0 else 0},
            "new_regime": {"totalTax": new_total, "effectiveRate": (new_total/income*100) if income > 0 else 0},
            "recommendation": recommendation
        }

    @staticmethod
    def check_gst_compliance(turnover: float, is_service: bool, state: str) -> dict:
        """Check GST registration requirement."""
        is_special = state in SPECIAL_STATES

        if is_service:
            threshold = 1000000 if is_special else 2000000
        else:
            threshold = 2000000 if is_special else 4000000

        required = turnover > threshold

        return {
            "registrationRequired": required,
            "threshold": threshold,
            "limitDescription": f"₹{threshold/100000:.0f} Lakhs ({'Special' if is_special else 'Regular'} State)",
            "recommendedAction": f"{'Register for GST immediately' if required else f'No registration needed. Headroom: ₹{(threshold-turnover)/100000:.1f}L'}"
        }

    @staticmethod
    def check_presumptive(gross_receipts: float, business_type: str) -> dict:
        """Check 44AD/44ADA eligibility."""
        is_professional = business_type == "professional"
        section = "44ADA" if is_professional else "44AD"
        limit = 7500000 if is_professional else 30000000

        eligible = gross_receipts <= limit

        if is_professional:
            deemed_income = gross_receipts * 0.50
            explanation = f"50% of ₹{gross_receipts:,.0f} = ₹{deemed_income:,.0f}"
        else:
            deemed_income = gross_receipts * 0.06
            explanation = f"6% of ₹{gross_receipts:,.0f} = ₹{deemed_income:,.0f}"

        return {
            "section": section,
            "eligible": eligible,
            "deemedIncome": deemed_income,
            "explanation": explanation
        }

    @staticmethod
    def get_deadlines(has_gst: bool = False) -> dict:
        """Get upcoming tax deadlines."""
        today = datetime.now()
        fy_year = today.year if today.month >= 4 else today.year - 1

        deadlines = [
            {"name": "Advance Tax Q4", "date": f"{fy_year+1}-03-15", "category": "Income Tax"},
            {"name": "ITR Filing", "date": f"{fy_year+1}-07-31", "category": "Income Tax"},
        ]

        if has_gst:
            next_month = (today.month % 12) + 1
            next_year = today.year + (1 if today.month == 12 else 0)
            deadlines.append({"name": "GSTR-3B", "date": f"{next_year}-{next_month:02d}-20", "category": "GST"})
            deadlines.append({"name": "GSTR-1", "date": f"{next_year}-{next_month:02d}-11", "category": "GST"})

        for d in deadlines:
            deadline_date = datetime.strptime(d["date"], "%Y-%m-%d")
            d["daysUntil"] = (deadline_date - today).days
            d["urgency"] = "CRITICAL" if d["daysUntil"] <= 7 else "WARNING" if d["daysUntil"] <= 15 else "NORMAL"

        return {"upcoming_deadlines": sorted(deadlines, key=lambda x: x["daysUntil"]), "total_count": len(deadlines)}

tools = TaxAllyTools()
print("✅ TaxAlly Tools initialized!")

✅ TaxAlly Tools initialized!


In [18]:
#@title 5️⃣ Agent Core - LLM Response Generator
import json
import re

SYSTEM_PROMPT = """You are TaxAlly, an expert AI tax compliance assistant for Indian individuals and micro-businesses.

You have access to these tools:
- calculate_income_tax(income, deductions_80c, deductions_80d): Compare old vs new tax regime
- check_gst_compliance(turnover, is_service, state): Check GST registration requirement
- check_presumptive(gross_receipts, business_type): Check 44AD/44ADA eligibility
- get_deadlines(has_gst): Get upcoming tax deadlines

When you need to use a tool, output it in this format:
```tool
{"tool": "tool_name", "params": {"param1": value1}}
```

Guidelines:
- Be precise and cite specific sections/rules
- Always explain your reasoning
- Flag compliance risks clearly
- Recommend professional consultation for complex cases
"""

def generate_response(user_message: str, profile: dict = None, history: list = None) -> dict:
    """Generate response with tool calling support."""

    # Build messages
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]

    if profile:
        profile_str = f"\nUser Profile: {json.dumps(profile)}"
        messages[0]["content"] += profile_str

    if history:
        for h in history[-5:]:  # Last 5 exchanges
            user_content = h.get("user", "")
            assistant_content = h.get("assistant", "")
            # Ensure strings (handle dict if passed)
            if isinstance(user_content, dict):
                user_content = json.dumps(user_content)
            if isinstance(assistant_content, dict):
                assistant_content = json.dumps(assistant_content)
            messages.append({"role": "user", "content": str(user_content)})
            messages.append({"role": "assistant", "content": str(assistant_content)})

    messages.append({"role": "user", "content": str(user_message)})

    # Format for Qwen
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

    # Execute tool calls
    tool_calls = []
    tool_pattern = r"```tool\n(.*?)\n```"
    matches = re.findall(tool_pattern, response_text, re.DOTALL)

    for match in matches:
        try:
            tool_data = json.loads(match)
            tool_name = tool_data.get("tool")
            params = tool_data.get("params", {})

            # Execute tool
            if tool_name == "calculate_income_tax":
                result = tools.calculate_income_tax(**params)
            elif tool_name == "check_gst_compliance":
                result = tools.check_gst_compliance(**params)
            elif tool_name == "check_presumptive":
                result = tools.check_presumptive(**params)
            elif tool_name == "get_deadlines":
                result = tools.get_deadlines(**params)
            else:
                result = {"error": f"Unknown tool: {tool_name}"}

            tool_calls.append({"tool": tool_name, "params": params, "result": result})
            # Inject result back into response
            response_text = response_text.replace(
                f"```tool\n{match}\n```",
                f"\n**Tool Result ({tool_name}):** {json.dumps(result)}\n"
            )
        except json.JSONDecodeError:
            continue

    return {
        "response": response_text,
        "tool_calls": tool_calls
    }

print("✅ Agent core ready!")

✅ Agent core ready!


In [19]:
#@title 6️⃣ FastAPI Server - Matches Backend Contracts
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Optional, List, Dict, Any
from datetime import datetime
import uuid

app = FastAPI(title="TaxAlly API", version="2.0.0")

# CORS for webapp
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# REQUEST/RESPONSE MODELS (Match backend contracts)
class ChatRequest(BaseModel):
    message: str
    session_id: Optional[str] = None
    user_id: Optional[str] = None
    profile: Optional[Dict[str, Any]] = None

class ChatResponse(BaseModel):
    response: str
    session_id: str
    tool_calls: List[dict] = []
    suggestions: List[str] = []

class ToolRequest(BaseModel):
    tool: str
    params: Dict[str, Any] = {}

# SESSION STORAGE
sessions: Dict[str, List[Dict]] = {}

# ENDPOINTS
@app.get("/health")
async def health():
    """Health check - matches backend's TaxAllyClient.getHealth() expectation"""
    return {
        "status": "healthy",
        "version": "2.0.0",
        "model": "Qwen/Qwen2.5-7B-Instruct",
        "timestamp": datetime.now().isoformat()
    }

@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """Chat endpoint - matches backend's TaxAllyClient.chat() contract"""
    session_id = request.session_id or str(uuid.uuid4())

    # Get/create session history
    history = sessions.get(session_id, [])

    # Generate response
    result = generate_response(request.message, request.profile, history)

    # Update history
    history.append({
        "user": request.message,
        "assistant": result["response"]
    })
    sessions[session_id] = history[-10:]  # Keep last 10

    return ChatResponse(
        response=result["response"],
        session_id=session_id,
        tool_calls=result.get("tool_calls", []),
        suggestions=["Ask about GST", "Calculate tax", "Check deadlines"]
    )

@app.get("/tools")
async def list_tools():
    """List available tools"""
    return {
        "tools": [
            {
                "name": "calculate_income_tax",
                "description": "Compare old vs new tax regime"
            },
            {
                "name": "check_gst_compliance",
                "description": "Check GST registration requirement"
            },
            {
                "name": "check_presumptive",
                "description": "Check 44AD/44ADA eligibility"
            },
            {
                "name": "get_deadlines",
                "description": "Get upcoming tax deadlines"
            }
        ],
        "count": 4
    }

@app.post("/tools/execute")
async def execute_tool(request: ToolRequest):
    """Execute a specific tool - matches backend's TaxAllyClient.executeTool() contract"""
    try:
        if request.tool == "calculate_income_tax":
            result = tools.calculate_income_tax(**request.params)
        elif request.tool == "check_gst_compliance":
            result = tools.check_gst_compliance(**request.params)
        elif request.tool == "check_presumptive":
            result = tools.check_presumptive(**request.params)
        elif request.tool == "get_deadlines":
            result = tools.get_deadlines(**request.params)
        else:
            raise HTTPException(
                status_code=400,
                detail=f"Unknown tool: {request.tool}"
            )

        return {
            "success": True,
            "tool": request.tool,
            "result": result
        }
    except Exception as e:
        return {
            "success": False,
            "tool": request.tool,
            "error": str(e)
        }

@app.get("/deadlines")
async def get_deadlines(has_gst: bool = False):
    """Get upcoming deadlines"""
    return tools.get_deadlines(has_gst)

print("✅ FastAPI app created!")

✅ FastAPI app created!


In [20]:
import subprocess
from pyngrok import ngrok
import os
import signal

# Kill all processes on port 8000
try:
    result = subprocess.run(
        ['fuser', '-k', '8000/tcp'],
        capture_output=True,
        text=True
    )
    print(f"✅ Killed processes on port 8000: {result.stdout}")
except Exception as e:
    print(f"ℹ️ fuser command failed: {e}")

# Alternative: use lsof and kill
try:
    result = subprocess.run(
        ['lsof', '-ti:8000'],
        capture_output=True,
        text=True
    )
    pids = result.stdout.strip().split('\n')
    for pid in pids:
        if pid:
            try:
                os.kill(int(pid), signal.SIGKILL)
                print(f"✅ Killed process {pid}")
            except:
                pass
except Exception as e:
    print(f"ℹ️ lsof command not available: {e}")

# Close ngrok tunnels
try:
    ngrok.kill()
    print("✅ Closed ngrok tunnels")
except Exception as e:
    print(f"ℹ️ No ngrok tunnels to close: {e}")

print("\n✅ Cleanup complete! Wait 2 seconds before running cell 8...")
import time
time.sleep(2)

✅ Killed processes on port 8000: 
✅ Closed ngrok tunnels

✅ Cleanup complete! Wait 2 seconds before running cell 8...


In [ ]:
#@title 8️⃣ 🚀 Start Server (RUN THIS!)
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import asyncio

nest_asyncio.apply()

# Start ngrok tunnel
tunnel = ngrok.connect(8000)
public_url = tunnel.public_url

print("=" * 60)
print("🎉 TaxAlly Server is LIVE!")
print("=" * 60)
print(f"\n🌐 PUBLIC URL: {public_url}")
print(f"\n📋 Add this to your webapp server/.env:")
print(f"\n   TAXALLY_API_URL={public_url}")
print(f"   USE_TAXALLY_SERVER=true")
print("\n" + "=" * 60)
print(f"\n🧪 Test it: {public_url}/health")
print("\n" + "=" * 60)
print("\n⏳ Server running... (Keep this cell running!)")
print("   Press STOP button to shut down.\n")

# Run server with Colab's event loop
config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)
await server.serve()

🎉 TaxAlly Server is LIVE!

🌐 PUBLIC URL: https://5f4b05fbe39c.ngrok-free.app

📋 Add this to your webapp server/.env:

   TAXALLY_API_URL=https://5f4b05fbe39c.ngrok-free.app
   USE_TAXALLY_SERVER=true


🧪 Test it: https://5f4b05fbe39c.ngrok-free.app/health


⏳ Server running... (Keep this cell running!)
   Press STOP button to shut down.



INFO:     Started server process [10240]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     103.233.93.167:0 - "GET /health HTTP/1.1" 200 OK
INFO: